<a href="https://colab.research.google.com/github/gimesia/MedicalImageSegmentation/blob/main/MISA_UNET_Tutorial_1_Introduction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


**Tutorial # 1 - Introduction**

Idea:

1.   Train, validate, and test a segmentation network using keras
2.   Test effect of skip connections (segnet -> unet)
3.   Test effect of hyperparameters (batch size, patch size, strides, epochs, # kernels) -> nn-unet
4.   Test effect of intensity standardisation
5.   Deal with limited resources (hardware and images; patch-wise processing)
6.   Get used to keras

Data: https://drive.google.com/drive/folders/1jKee0ry5sR_mvc2MvYmvBlOUF0fJ7suz?usp=drive_link

**Import libraries**

In [1]:
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt

import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

**Mount drive**

In [2]:
from google.colab import drive
drive.mount("/content/drive/")

Mounted at /content/drive/


**Define parameters**

In [3]:
import os
DATA_PATH = "/content/drive/MyDrive/Colab Notebooks/OASIS_subset_sorted"

In [4]:
# dataset parameters
FNAME_PATTERN = f'{DATA_PATH}/OAS2_{0:04d}_MR1/OAS2_{0:04d}_MR1{1:}.nii.gz'
IMAGE_SIZE = (128, 256, 256)

# network parameters
N_CLASSES = 4
N_INPUT_CHANNELS = 1
SCALING_FACTOR = 1
PATCH_SIZE = (32, 32)
PATCH_STRIDE = (32, 32)

# training, validation, test parameters
TRAINING_VOLUMES = [2, 7, 9, 10, 12, 13, 14, 16, 18]
VALIDATION_VOLUMES = [21, 22, 26]
TEST_VOLUMES = [20] # Difficult volumes: 4, 5, 8, 12, 17, 23, 28

# data preparation parameters
CONTENT_THRESHOLD = 0.3

# training parameters
N_EPOCHS = 20
BATCH_SIZE = 32
PATIENCE = 10
MODEL_FOLDER = 'checkpoint.keras'
OPTIMISER = 'Adam'
LOSS = 'categorical_crossentropy'

**Define SegNet architecture**

In [5]:
def get_segnet(img_size=PATCH_SIZE, n_classes=N_CLASSES, n_input_channels=N_INPUT_CHANNELS, scaling_factor=SCALING_FACTOR):
    inputs = keras.Input(shape=img_size + (n_input_channels, ))

    # Encoding path
    conv1 = layers.Conv2D(32*scaling_factor, (3, 3), padding="same", activation='relu')(inputs)
    max1 = layers.MaxPooling2D((2, 2))(conv1)

    conv2 = layers.Conv2D(64*scaling_factor, (3, 3), padding="same", activation='relu')(max1)
    max2 = layers.MaxPooling2D((2, 2))(conv2)

    conv3 = layers.Conv2D(128*scaling_factor, (3, 3), padding="same", activation='relu')(max2)
    max3 = layers.MaxPooling2D((2, 2))(conv3)

    lat = layers.Conv2D(256*scaling_factor, (3, 3), padding="same", activation='relu')(max3)

    # Decoding path
    # Add concatenation layers later on to create U-Nets
    # example: cat = layers.Concatenate()([layer_1, layer_2])
    # we could use additions instead of concatenations too
    # example: cat = layers.Add()([layer_1, layer_2])

    up1 = layers.UpSampling2D((2, 2))(lat)
    conv4 = layers.Conv2D(128*scaling_factor, (3, 3), padding="same", activation='relu')(up1)

    up2 = layers.UpSampling2D((2, 2))(conv4)
    conv5 = layers.Conv2D(64*scaling_factor, (3, 3), padding="same", activation='relu')(up2)

    up3 = layers.UpSampling2D((2, 2))(conv5)
    conv6 = layers.Conv2D(32*scaling_factor, (3, 3), padding="same", activation='relu')(up3)

    outputs = layers.Conv2D(n_classes, (1, 1), activation="softmax")(conv6)

    model = keras.Model(inputs, outputs)

    return model

**Load data**

In [18]:
# Correcting FNAME_PATTERN
FNAME_PATTERN = f'{DATA_PATH}/OAS2_{0:04d}_MR1/OAS2_{0:04d}_MR1{1}.nii.gz'

# Correcting pattern lambda
pattern = lambda x, y: f'{DATA_PATH}/OAS2_{x:04d}_MR1/OAS2_{x:04d}_MR1{y}.nii.gz'

def load_data(volume_list, image_size=IMAGE_SIZE, fname_pattern=pattern):
  n_volumes = len(volume_list)
  T1_volumes = np.zeros((n_volumes, *image_size, 1))
  labels = np.zeros((n_volumes, *image_size, 1))
  for iFile, iID in enumerate(volume_list):
    print(iID)
    print(fname_pattern(iID, '_BrainExtractionBrain'))

    img_data = nib.load(fname_pattern(iID, '_BrainExtractionBrain'))
    T1_volumes[iFile, ..., 0] = np.transpose(img_data.get_fdata(), (2, 0, 1))

    seg_data = nib.load(fname_pattern(iID, '_Segmentation'))
    labels[iFile, ..., 0] = np.transpose(seg_data.get_fdata(), (2, 0, 1))

  return (T1_volumes, labels)

In [19]:
pattern(21, "__DEST")

'/content/drive/MyDrive/Colab Notebooks/OASIS_subset_sorted/OAS2_0021_MR1/OAS2_0021_MR1__DEST.nii.gz'

In [20]:
(training_volumes_T1, training_labels) = load_data(TRAINING_VOLUMES)
(validation_volumes_T1, validation_labels) = load_data(VALIDATION_VOLUMES)
(testing_volumes_T1, testing_labels) = load_data(TEST_VOLUMES)

2
/content/drive/MyDrive/Colab Notebooks/OASIS_subset_sorted/OAS2_0002_MR1/OAS2_0002_MR1_BrainExtractionBrain.nii.gz
7
/content/drive/MyDrive/Colab Notebooks/OASIS_subset_sorted/OAS2_0007_MR1/OAS2_0007_MR1_BrainExtractionBrain.nii.gz
9
/content/drive/MyDrive/Colab Notebooks/OASIS_subset_sorted/OAS2_0009_MR1/OAS2_0009_MR1_BrainExtractionBrain.nii.gz
10
/content/drive/MyDrive/Colab Notebooks/OASIS_subset_sorted/OAS2_0010_MR1/OAS2_0010_MR1_BrainExtractionBrain.nii.gz
12
/content/drive/MyDrive/Colab Notebooks/OASIS_subset_sorted/OAS2_0012_MR1/OAS2_0012_MR1_BrainExtractionBrain.nii.gz
13
/content/drive/MyDrive/Colab Notebooks/OASIS_subset_sorted/OAS2_0013_MR1/OAS2_0013_MR1_BrainExtractionBrain.nii.gz
14
/content/drive/MyDrive/Colab Notebooks/OASIS_subset_sorted/OAS2_0014_MR1/OAS2_0014_MR1_BrainExtractionBrain.nii.gz
16
/content/drive/MyDrive/Colab Notebooks/OASIS_subset_sorted/OAS2_0016_MR1/OAS2_0016_MR1_BrainExtractionBrain.nii.gz
18
/content/drive/MyDrive/Colab Notebooks/OASIS_subset_sort

**Pre-process data**

In [ ]:
plt.hist(training_volumes_T1[training_labels>0].flatten(), 100, label='training', alpha=0.5, density=True)
plt.hist(validation_volumes_T1[validation_labels>0].flatten(), 100, label='validation', alpha=0.5, density=True)
plt.hist(testing_volumes_T1[testing_labels>0].flatten(), 100, label='test', alpha=0.5, density=True)
plt.legend(loc='upper right')
plt.show()

NameError: name 'training_volumes_T1' is not defined

In [ ]:
# Space for trying intensity standardisation strategies


In [ ]:
plt.hist(training_volumes_T1[training_labels>0].flatten(), 100, label='training', alpha=0.5, density=True)
plt.hist(validation_volumes_T1[validation_labels>0].flatten(), 100, label='validation', alpha=0.5, density=True)
plt.hist(testing_volumes_T1[testing_labels>0].flatten(), 100, label='test', alpha=0.5, density=True)
plt.legend(loc='upper right')
plt.show()

**Extract *useful* patches**

This step is fundamental, we want to provide the network with useful information

In [ ]:
def extract_patches(x, patch_size, patch_stride) :
  return tf.image.extract_patches(
    x,
    sizes=[1, *patch_size, 1],
    strides=[1, *patch_stride, 1],
    rates=[1, 1, 1, 1],
    padding='SAME', name=None)

In [ ]:
def extract_useful_patches(
    volumes, labels,
    image_size=IMAGE_SIZE,
    patch_size=PATCH_SIZE,
    stride=PATCH_STRIDE,
    threshold=CONTENT_THRESHOLD,
    num_classes=N_CLASSES) :
  volumes = volumes.reshape([-1, image_size[1], image_size[2], 1])
  labels = labels.reshape([-1, image_size[1], image_size[2], 1])

  vol_patches = extract_patches(volumes, patch_size, stride).numpy()
  seg_patches = extract_patches(labels, patch_size, stride).numpy()

  vol_patches = vol_patches.reshape([-1, *patch_size, 1])
  seg_patches = seg_patches.reshape([-1, *patch_size, ])

  foreground_mask = seg_patches != 0

  useful_patches = foreground_mask.sum(axis=(1, 2)) > threshold * np.prod(patch_size)

  vol_patches = vol_patches[useful_patches]
  seg_patches = seg_patches[useful_patches]

  seg_patches = tf.keras.utils.to_categorical(
    seg_patches, num_classes=N_CLASSES)

  return (vol_patches, seg_patches)

In [ ]:
# extract patches from training set
(training_patches_T1, training_patches_seg) = extract_useful_patches(training_volumes_T1, training_labels)

# extract patches from validation set
(validation_patches_T1, validation_patches_seg) = extract_useful_patches(validation_volumes_T1, validation_labels)

NameError: name 'training_volumes_T1' is not defined

**Instantiate SegNet model and train it**

*   When/how do we stop training?
*   Should we use validation split in keras?
*   How large should the batch size be?

In [ ]:
segnet = get_segnet()
segnet.compile(optimizer=OPTIMISER, loss=LOSS)
h = segnet.fit(
    x=training_patches_T1,
    y=training_patches_seg,
    validation_data=(validation_patches_T1, validation_patches_seg),
    batch_size=BATCH_SIZE,
    epochs=N_EPOCHS,
    verbose=1)

We could stop training when validation loss increases substantially

In [ ]:
plt.figure()
plt.plot(range(N_EPOCHS), h.history['loss'], label='loss')
plt.plot(range(N_EPOCHS), h.history['val_loss'], label='val_loss')
plt.legend()
plt.show()

NameError: name 'h' is not defined

<Figure size 640x480 with 0 Axes>

Using callbacks to stop training and avoid overfitting


*   Early stopping with a certain patience
*   Save (and load!) best model



In [ ]:
my_callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=PATIENCE),
    tf.keras.callbacks.ModelCheckpoint(filepath=MODEL_FOLDER, save_best_only=True)
]

segnet = get_segnet()
segnet.compile(optimizer=OPTIMISER, loss=LOSS)
h = segnet.fit(
    x=training_patches_T1,
    y=training_patches_seg,
    validation_data=(validation_patches_T1, validation_patches_seg),
    batch_size=BATCH_SIZE,
    epochs=N_EPOCHS,
    callbacks=my_callbacks,
    verbose=1)

NameError: name 'training_patches_T1' is not defined

**Load best model**

In [ ]:
segnet = get_segnet(
    img_size=(IMAGE_SIZE[1], IMAGE_SIZE[2]),
    n_classes=N_CLASSES,
    n_input_channels=N_INPUT_CHANNELS)

segnet.compile(optimizer=OPTIMISER, loss=LOSS)
segnet.load_weights(MODEL_FOLDER)

**Prepare test data**

In [ ]:
testing_volumes_T1_processed = testing_volumes_T1.reshape([-1, IMAGE_SIZE[1], IMAGE_SIZE[2], 1])
testing_labels_processed = testing_labels.reshape([-1, IMAGE_SIZE[1], IMAGE_SIZE[2]])

**Predict labels for test data**

In [ ]:
prediction = segnet.predict(x=testing_volumes_T1_processed)

prediction = np.argmax(prediction, axis=3)

plt.imshow(prediction[:, :, 128])

In [ ]:
!pip install medpy

In [ ]:
import numpy as np
import nibabel as nib
from medpy.metric.binary import dc, hd, ravd

def compute_dice(prediction, reference) :
  for c in np.unique(reference) :
    dsc_val = dc(prediction == c, reference==c)
    print(f'Dice coefficient class {c} equal to {dsc_val : .2f}')

def compute_hd(prediction, reference, voxel_spacing) :
  for c in np.unique(prediction) :
    hd_val = hd(prediction == c, reference==c, voxelspacing=voxel_spacing, connectivity=1)
    print(f'Hausdorff distance class {c} equal to {hd_val : .2f}')

def compute_ravd(prediction, reference) :
  for c in np.unique(prediction) :
    ravd_val = ravd(prediction == c, reference==c)
    print(f'RAVD coefficient class {c} equal to {ravd_val : .2f}')

compute_dice(prediction, testing_labels_processed) # the higher -> the better
compute_hd(prediction, testing_labels_processed, [1, 1, 1]) # the lower -> the better
compute_ravd(prediction, testing_labels_processed) # the closer to zero -> the better